In [ ]:
# Authorize me
from google.colab import auth
auth.authenticate_user()

In [ ]:
## Check Dedalus isn't already installed
## Installation takes about 1-2 mins

try:
    import dedalus.public as de
    print("Dedalus already installed")
except:
    print("Dedalus not installed yet. Let's do it.")

# Step 1: Install FFTW
!apt-get install libfftw3-dev
!apt-get install libfftw3-mpi-dev

# Step 2: Set paths for Dedalus installation
import os
os.environ['MPI_INCLUDE_PATH'] = "/usr/lib/x86_64-linux-gnu/openmpi/include"
os.environ['MPI_LIBRARY_PATH'] = "/usr/lib/x86_64-linux-gnu"
os.environ['FFTW_INCLUDE_PATH'] = "/usr/include"
os.environ['FFTW_LIBRARY_PATH'] = "/usr/lib/x86_64-linux-gnu"

# Step 3: Install Dedalus using pip
!pip3 install dedalus

# Step 4: Check Dedalus is importable
print()
print()
try:
    import dedalus.public as de
    print("Dedalus successfully installed :)")
except:
    print("Error installing Dedalus :(")
    raise

Dedalus not installed yet. Let's do it.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libfftw3-bin libfftw3-double3 libfftw3-long3 libfftw3-quad3 libfftw3-single3
Suggested packages:
  libfftw3-doc
The following NEW packages will be installed:
  libfftw3-bin libfftw3-dev libfftw3-double3 libfftw3-long3 libfftw3-quad3 libfftw3-single3
0 upgraded, 6 newly installed, 0 to remove and 45 not upgraded.
Need to get 4,654 kB of archives.
After this operation, 24.7 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libfftw3-double3 amd64 3.3.8-2ubuntu8 [770 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libfftw3-long3 amd64 3.3.8-2ubuntu8 [335 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 libfftw3-quad3 amd64 3.3.8-2ubuntu8 [614 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/main amd64 libfftw3-single3 amd64 3.3.8-2

2024-05-06 13:47:57,804 dedalus 0/1 WARNING :: Threading has not been disabled. This may massively degrade Dedalus performance.


2024-05-06 13:47:57,808 dedalus 0/1 WARNING :: We strongly suggest setting the "OMP_NUM_THREADS" environment variable to "1".


INFO:numexpr.utils:NumExpr defaulting to 8 threads.


2024-05-06 13:47:57,905 numexpr.utils 0/1 INFO :: NumExpr defaulting to 8 threads.


DEBUG:h5py._conv:Creating converter from 7 to 5
DEBUG:h5py._conv:Creating converter from 5 to 7
DEBUG:h5py._conv:Creating converter from 7 to 5
DEBUG:h5py._conv:Creating converter from 5 to 7


Dedalus successfully installed :)


In [ ]:
"""
Dedalus script simulating the 1D Korteweg-de Vries / Burgers equation.
This script demonstrates solving a 1D initial value problem and produces
a space-time plot of the solution. It should take just a few seconds to
run (serial only).

We use a Fourier basis to solve the IVP:
    dt(u) + u*dx(u) = a*dx(dx(u)) + b*dx(dx(dx(u)))

To run and plot:
    $ python3 kdv_burgers.py
"""

import numpy as np
import matplotlib.pyplot as plt
import dedalus.public as d3
import logging
logger = logging.getLogger(__name__)

# geo_start = 0
# geo_end = 1
# time_start = 0
# time_end = 1
def num_solver(
        geo_start,
        geo_end,
        time_start,
        time_end,
    ):

    # Parameters
    # TODO set PINNS from 0,
    Lx = geo_end


    # TODO align with PINNS
    # Compare values of the solution on these points from Deadalus
    # Calculate error with the PINNS test data
    num_basis_modes = 256 # num_basis modes 2048

    # viscosity term
    a = 1e-2 / np.pi
    # set b = 0 is Burgers b = 2e-4, This is k in PINNs
    b = 0
    # padding with zeros (ratio)
    dealias = 3/2

    # same as PINNS stop time
    stop_sim_time = time_end

    # TODO: what is this?
    timestepper = d3.SBDF2
    # runs 500 * 10 = 5k
    # unit of time is 500 steps
    #
    timestep = 4*(1e-4)
    dtype = np.float64

    # Bases
    # TODO 1D?
    coordinates = d3.Coordinate('x')

    # cart_coordinates = d3.CartesianCoordinates('x')
    # cart_dist = d3.Distributor(cart_coordinates, dtype=dtype)
    # cart_basis = d3.Ca(
    #     cart_dist,
    #     size=1,
    #     bounds=(0, Lx)
    # )

    dist = d3.Distributor(coordinates, dtype=dtype)

    # TODO: Nx? Num points? Num F modes
    # bounds
    xbasis = d3.RealFourier(
        coordinates,
        size=num_basis_modes, # number of modes for the basis
        bounds=(0, Lx),
        dealias=dealias,
    )

    # Fields
    u = dist.Field(name='u', bases=xbasis)

    # Substitutions
    dx = lambda A: d3.Differentiate(A, coordinates)

    # Problem
    # Initial value problem
    problem = d3.IVP([u], namespace=locals())

    problem.add_equation("dt(u) - a*dx(dx(u)) - b*dx(dx(dx(u))) = - u*dx(u)")

    # Initial conditions
    x = dist.local_grid(xbasis)
    # n = 20
    # u['g'] = np.log(1 + np.cosh(n)**2/np.cosh(n*(x-0.2*Lx))**2) / (2*n)
    # u['g'] = -np.sin(8 * np.pi * x), # -sin(8*pi*x)
    # Be aware this needs to be adjusted to match PINNs
    u['g'] = -np.sin(np.pi * x)
    # u['g'] = initial_condition_fn

    # Solver
    solver = problem.build_solver(timestepper)
    solver.stop_sim_time = stop_sim_time

    # Main loop
    u.change_scales(1)
    u_list = [np.copy(u['g'])]
    t_list = [solver.sim_time]
    while solver.proceed:
        solver.step(timestep)
        # if solver.iteration % 100 == 0:
        #     logger.info('Iteration=%i, Time=%e, dt=%e' %(solver.iteration, solver.sim_time, timestep))
        if solver.iteration % 25 == 0:
            u.change_scales(1)
            u_list.append(np.copy(u['g']))
            t_list.append(solver.sim_time)

    return x, u, u_list, t_list


def fit_solver(x, u, u_list, t_list):
    u_list_np = np.array(u_list)
    u_list_np = u_list_np[:len(u_list_np)-1]
    t_list_np = np.array(t_list)
    t_list_np = t_list_np[:len(t_list_np)-1]
    xx, tt = np.meshgrid(x, t_list_np)
    X = np.vstack((np.ravel(xx), np.ravel(tt))).T
    y = u_list_np.flatten()[:, None]
    return X, y


In [ ]:
!pip install deepxde
!pip install numpy==1.19.5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.2/182.2 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.7/107.7 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 23.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for numpy (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for numpy
Failed to build numpy
ERROR: Could not build wheels for numpy, which is required to install pyproject.toml-based projects


In [ ]:
import deepxde as dde
from deepxde.backend import tf

No backend selected.
Finding available backend...


Level 1:tensorflow:Registering Relayout (<function _relayout_gradient at 0x7a2ce3239360>) in gradient.
Level 1:tensorflow:Registering RelayoutLike (<function _relayout_grad_gradient at 0x7a2ce30b1990>) in gradient.
Level 1:tensorflow:Registering CopyToMesh (<function _copy_to_mesh_gradient at 0x7a2ce3239510>) in gradient.
Level 1:tensorflow:Registering CopyToMeshGrad (<function _copy_to_mesh_grad_gradient at 0x7a2ce30b1bd0>) in gradient.
Level 1:tensorflow:Registering FakeQuantWithMinMaxArgs (<function _FakeQuantWithMinMaxArgsGradient at 0x7a2ce3014c10>) in gradient.
Level 1:tensorflow:Registering FakeQuantWithMinMaxVars (<function _FakeQuantWithMinMaxVarsGradient at 0x7a2ce3086200>) in gradient.
Level 1:tensorflow:Registering FakeQuantWithMinMaxVarsPerChannel (<function _FakeQuantWithMinMaxVarsPerChannelGradient at 0x7a2ce30863b0>) in gradient.
Level 1:tensorflow:Registering QuantizeAndDequantizeV4 (<function _QuantizeAndDequantizeV4Grad at 0x7a2ce3014e50>) in gradient.
Level 1:tensor

Found tensorflow.compat.v1
Setting the default backend to "tensorflow.compat.v1". You can change it in the ~/.deepxde/config.json file or export the DDE_BACKEND environment variable. Valid options are: tensorflow.compat.v1, tensorflow, pytorch, jax, paddle (all lowercase)
2024-05-06 13:49:36,027 tensorflow 0/1 WARNING :: From /usr/local/lib/python3.10/dist-packages/tensorflow/python/compat/v2_compat.py:108: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
Instructions for updating:
non-resource variables are not supported in the long term


Level 1:tensorflow:Disabling resource variables
Level 1:tensorflow:Disabling tensor equality
Level 1:tensorflow:Disabling control flow v2
Level 1:tensorflow:Enabling v2 tensorshape
Enable just-in-time compilation with XLA.



In [ ]:
dde.backend.set_default_backend("tensorflow")
#tf.enable_eager_execution()
print(tf.__version__)
print(tf.executing_eagerly())

Setting the default backend to "tensorflow". You can change it in the ~/.deepxde/config.json file or export the DDE_BACKEND environment variable. Valid options are: tensorflow.compat.v1, tensorflow, pytorch, jax, paddle (all lowercase)
2.15.0
False


In [ ]:
import deepxde as dde
import numpy as np
from os.path import exists



# EVAL_DATA = "deepxde/examples/dataset/Burgers.npz"


# def precheck():
#     if not exists(EVAL_DATA):
#         raise FileNotFoundError("Evaluation data not loaded into notebook")

# def gen_testdata(eval_data):
#     data = np.load(eval_data)
#     t, x, exact = data["t"], data["x"], data["usol"].T
#     xx, tt = np.meshgrid(x, t)
#     X = np.vstack((np.ravel(xx), np.ravel(tt))).T
#     y = exact.flatten()[:, None]
#     return X, y


def pde(x, y):
    dy_x = dde.grad.jacobian(y, x, i=0, j=0)
    dy_t = dde.grad.jacobian(y, x, i=0, j=1)
    dy_xx = dde.grad.hessian(y, x, i=0, j=0)
    dy_xxxx = dde.grad.hessian(dy_xx, x, i=0, j=0)
    viscosity = 0.01 # TODO match viscostity with Deadalus
    k = 0
    return dy_t + y * dy_x + (k * dy_xxxx) + viscosity / np.pi * dy_xx # TODO add nudged terms.



def run_train_eval(
        train_test_contidions: list,
        iters=50_000,
        original_eval=False,
    ):

    """
    Run train and evaluation

    Ags:
    - train_test_conditions: a list of dictionaries to train and evaluate
      our experiments. [
        {
            "train_time": [0.0, 0.99] # the time start/end to train the model, float
            "train_geo": [-1,1] # initial geo interval for train
            "test_time": [0.5,0.75] # the time start to test the model, float
            "test_geo": [-1, 1], # initial geo interval for test
        },
        {
            ...
        }
      ]
    - iterations: int, number of epochs to train the model

    Returns:
    - result: a list of dictionaries with the training results

    Raises:
    - FileNotFoundError if test data not on the box
    """

    # dummy check
    # precheck()



    # Set the time domain to be less than the test data

    # timedomain = dde.geometry.TimeDomain(0.0, 0.99)
    # TODO:
    # - run on different subsets of time where t!=0, ie 0.25:0.75
    # - regular intervals of certain length
    # - random intervals of time (how to randomly choose disjoint intervals)
    #   - sparse and small
    # - outcome: corelations of length and how sparsely we are in the domin with the error
    # - investigate other metrics?
    # - compare errors directly over the training intervals vs outside the intervals
    # - perform the same for space
    # - break up space for fixed time

    result = []

    i = 1

    #iterate over different time training
    for condition in train_test_contidions:
        print(f"iteration: {i}")

        # pull all the conditions
        train_time_start = condition.get("train_time_start")
        train_time_end = condition.get("train_time_end")
        train_geo_start = condition.get("train_geo_start")
        train_geo_end = condition.get("train_geo_end")
        test_time_start = condition.get("test_time_start")
        test_time_end = condition.get("test_time_end")
        test_geo_start = condition.get("test_geo_start")
        test_geo_end = condition.get("test_geo_end")

        geom = dde.geometry.Interval(train_geo_start, train_geo_end)
        timedomain = dde.geometry.TimeDomain(train_time_start, train_time_end)
        geomtime = dde.geometry.GeometryXTime(geom, timedomain)

        # TODO: is this correct?
        # icbc = initial condition Dirichlet boundry condition
        bc = dde.icbc.DirichletBC(
            geomtime,
            lambda x: 0,
            lambda _,
            on_boundary:
            on_boundary
        )

        ic = dde.icbc.IC(
            geomtime,
            lambda x: -np.sin(np.pi * x[:, 0:1]),
            #lambda x: -np.sin(8 * np.pi * x[:, 0:1]), # -sin(8*pi*x)
            lambda _,
            on_initial: on_initial
        )

        # TODO any changes around domain and boundries?
        # TODO: do we need the data to be at least at this resolution?
        data = dde.data.TimePDE(
            geomtime, # TODO how is it generating data for training
            pde,
            [bc, ic],
            num_domain=2540,
            num_boundary=80,
            num_initial=160,
            # num_test=100,
        )

        net = dde.nn.FNN([2] + [20] * 3 + [1], "tanh", "Glorot normal")
        model = dde.Model(data, net)

        model.compile("adam", lr=1e-3)
        model.train(iterations=iters, display_every=10000)
        model.compile("L-BFGS")
        losshistory, train_state = model.train()

        # test against full timedomain in test data from numerical solver?

        if original_eval:
            pass
            # X, y_true = gen_testdata(EVAL_DATA)

        else:
            x_d, u_d, u_list, t_list = num_solver(
                test_geo_start,
                test_geo_end,
                test_time_start,
                test_time_end
                )
            X, y_true = fit_solver(x_d, u_d, u_list, t_list)


        y_pred = model.predict(X)

        # TODO, not sure we want to use this?
        f = model.predict(X, operator=pde)
        mean_residual = np.mean(np.absolute(f))



        rel_error = dde.metrics.l2_relative_error(y_true, y_pred)
        metric = condition
        metric.update({
            # "conditions": condition,
            "mean_residual": mean_residual,
            "l2_relative_error": rel_error,
            })


        print(metric)
        result.append(metric)
        i = i + 1

    return result



In [ ]:
loggers = [logging.getLogger(name) for name in logging.root.manager.loggerDict]
for logger in loggers:
    logger.setLevel(logging.ERROR)

In [ ]:
import random

import numpy as np




def generate_two_points(ranges):
    start = ranges[0]
    end = ranges[1]
    distance = abs(start - end)

    point_1 = np.random.uniform(start, end)
    if point_1 <= (distance/2):
        point_2 = np.random.uniform(point_1, end)
    else:
        point_2 = np.random.uniform(start, point_1)
    points = [round(point_1, 4), round(point_2, 4)]
    return sorted(points)


def generate_box(rangex, rangey):
    return generate_two_points(rangex) + generate_two_points(rangey)


def generate_train_test_boxes(rangex, rangey, overlap):
    train_box = generate_box(rangex, rangey)
    if overlap:
        test_box = generate_box(
            [train_box[0], train_box[1]],
             [train_box[2], train_box[3]])
    if not overlap:
        before = bool(random.getrandbits(1))
        if before:
            test_box = generate_box(
                [rangex[0], train_box[0]],
                [rangey[0], train_box[2]])
        else:
            test_box = generate_box(
                [train_box[1], rangex[1]],
                [train_box[2], rangey[1]])

    return train_box, test_box


# p1, p2 = generate_two_points([0, 1])
# print(f"box: {generate_box(range_space, range_time)}")
# print(f"Overlap: {generate_train_test_boxes(range_space, range_time)}")

# print(f"No Overlap: {generate_train_test_boxes(range_space, range_time, overlap=False)}")

# print(f"{p1}, {p2}")

In [ ]:
# box: [0.34, 0.8572, 0.1822, 0.8902]
# Overlap: ([0.4022, 0.7992, 0.1667, 0.4093], [0.4703, 0.5556, 0.2372, 0.2502])
# No Overlap: ([0.302, 0.595, 0.6769, 0.7547], [0.0854, 0.271, 0.0915, 0.2096])


import random

def generate_random_condition(rangex, rangey, overlap):

    boxes = generate_train_test_boxes(rangex, rangey, overlap)

    return {
        "train_geo_start": boxes[0][0],
        "train_geo_end": boxes[0][1],
        "train_time_start": boxes[0][2],
        "train_time_end": boxes[0][3],

        # test
        "test_geo_start": boxes[0][0],
        "test_geo_end": boxes[0][1],
        "test_time_start": boxes[0][2],
        "test_time_end": boxes[0][3],

        "overlap": overlap,
    }

def generate_multiple_conditions(n, rangex, rangey, overlap):
    return [generate_random_condition(rangex, rangey, overlap) for i in range(n)]


In [ ]:
import csv
import time

from google.cloud import storage

STORAGE_CLIENT = storage.Client()
BUCKET = STORAGE_CLIENT.get_bucket('pinns_gabe')


def save_csv(results, overlap):
    file_name = "results_" + str(int(time.time())) + "_.csv"
    with open(file_name, 'w+', newline='') as csvfile:
        fieldnames = ['train_time_start', 'train_time_end','train_geo_start', \
                      'train_geo_end','test_time_start', 'test_time_end', \
                      'test_geo_start', 'test_geo_end', 'mean_residual', \
                      'l2_relative_error', 'overlap']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for item in results:
            writer.writerow(item)

    blob = BUCKET.blob(file_name)
    blob.upload_from_filename(file_name)
    print(f"uploaded {file_name}")

    return file_name

In [ ]:
number_runs = 20
overlap = True
range_space, range_time = [0,1], [0,1]
conditions = generate_multiple_conditions(number_runs, range_space, range_time, overlap)
result = run_train_eval(conditions, overlap)


iteration: 1
Compiling model...
Building feed-forward neural network...
'build' took 0.063517 s



/usr/local/lib/python3.10/dist-packages/deepxde/nn/tensorflow_compat_v1/fnn.py:116: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  return tf.layers.dense(


'compile' took 7.717157 s

Training model...

Step      Train loss                        Test loss                         Test metric
0         [7.02e-02, 1.69e-01, 2.61e-01]    [7.02e-02, 1.69e-01, 2.61e-01]    []  
1         [3.95e-02, 1.74e-01, 2.45e-01]    [3.95e-02, 1.74e-01, 2.45e-01]    []  

Best model at step 1:
  train loss: 4.58e-01
  test loss: 4.58e-01
  test metric: []

'train' took 54.350181 s

Compiling model...
'compile' took 7.794020 s

Training model...

Step      Train loss                        Test loss                         Test metric
1         [3.95e-02, 1.74e-01, 2.45e-01]    [3.95e-02, 1.74e-01, 2.45e-01]    []  
1000      [8.27e-03, 1.23e-02, 7.84e-02]    [8.27e-03, 1.23e-02, 7.84e-02]        
2000      [7.58e-03, 8.63e-03, 7.77e-02]    [7.58e-03, 8.63e-03, 7.77e-02]        
2164      [7.49e-03, 8.85e-03, 7.73e-02]    [7.49e-03, 8.85e-03, 7.73e-02]    []  

Best model at step 2164:
  train loss: 9.37e-02
  test loss: 9.37e-02
  test metric: []

'train' 

/usr/local/lib/python3.10/dist-packages/deepxde/nn/tensorflow_compat_v1/fnn.py:116: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  return tf.layers.dense(


'compile' took 7.730872 s

Training model...

Step      Train loss                        Test loss                         Test metric
0         [7.76e-02, 3.37e-02, 4.02e-01]    [7.76e-02, 3.37e-02, 4.02e-01]    []  
1         [7.77e-02, 5.05e-02, 3.43e-01]    [7.77e-02, 5.05e-02, 3.43e-01]    []  

Best model at step 1:
  train loss: 4.71e-01
  test loss: 4.71e-01
  test metric: []

'train' took 51.773008 s

Compiling model...
'compile' took 7.847697 s

Training model...

Step      Train loss                        Test loss                         Test metric
1         [7.77e-02, 5.05e-02, 3.43e-01]    [7.77e-02, 5.05e-02, 3.43e-01]    []  
1000      [4.24e-04, 2.79e-04, 3.03e-03]    [4.24e-04, 2.79e-04, 3.03e-03]        
2000      [2.19e-04, 1.64e-04, 2.58e-03]    [2.19e-04, 1.64e-04, 2.58e-03]        
2964      [1.78e-04, 1.70e-04, 2.33e-03]    [1.78e-04, 1.70e-04, 2.33e-03]    []  

Best model at step 2964:
  train loss: 2.68e-03
  test loss: 2.68e-03
  test metric: []

'train' 

...

'compile' took 9.404474 s

Training model...

Step      Train loss                        Test loss                         Test metric
0         [2.89e-02, 8.83e-02, 1.12e+00]    [2.89e-02, 8.83e-02, 1.12e+00]    []  
1         [1.51e-02, 5.95e-02, 1.02e+00]    [1.51e-02, 5.95e-02, 1.02e+00]    []  

Best model at step 1:
  train loss: 1.09e+00
  test loss: 1.09e+00
  test metric: []

'train' took 67.500584 s

Compiling model...
'compile' took 9.589298 s

Training model...

Step      Train loss                        Test loss                         Test metric
1         [1.51e-02, 5.95e-02, 1.02e+00]    [1.51e-02, 5.95e-02, 1.02e+00]    []  
1000      [4.81e-03, 1.84e-02, 1.12e-01]    [4.81e-03, 1.84e-02, 1.12e-01]        
2000      [3.50e-03, 2.05e-02, 9.95e-02]    [3.50e-03, 2.05e-02, 9.95e-02]        
2040      [3.58e-03, 2.04e-02, 9.95e-02]    [3.58e-03, 2.04e-02, 9.95e-02]    []  

Best model at step 2040:
  train loss: 1.23e-01
  test loss: 1.23e-01
  test metric: []

'train' 

/usr/local/lib/python3.10/dist-packages/deepxde/nn/tensorflow_compat_v1/fnn.py:116: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  return tf.layers.dense(


'compile' took 9.364757 s

Training model...

Step      Train loss                        Test loss                         Test metric
0         [2.74e-04, 6.05e-05, 7.34e-01]    [2.74e-04, 6.05e-05, 7.34e-01]    []  
1         [1.27e-03, 3.17e-03, 6.65e-01]    [1.27e-03, 3.17e-03, 6.65e-01]    []  

Best model at step 1:
  train loss: 6.69e-01
  test loss: 6.69e-01
  test metric: []

'train' took 74.028679 s

Compiling model...
'compile' took 9.824540 s

Training model...

Step      Train loss                        Test loss                         Test metric
1         [1.27e-03, 3.17e-03, 6.65e-01]    [1.27e-03, 3.17e-03, 6.65e-01]    []  
1000      [1.31e-02, 8.18e-02, 1.06e-01]    [1.31e-02, 8.18e-02, 1.06e-01]        
1599      [1.20e-02, 7.46e-02, 1.11e-01]    [1.20e-02, 7.46e-02, 1.11e-01]    []  

Best model at step 1599:
  train loss: 1.98e-01
  test loss: 1.98e-01
  test metric: []

'train' took 318.744994 s

{'train_geo_start': 0.3905, 'train_geo_end': 0.8616, 'train_time

/usr/local/lib/python3.10/dist-packages/deepxde/nn/tensorflow_compat_v1/fnn.py:116: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  return tf.layers.dense(


'compile' took 9.476394 s

Training model...

Step      Train loss                        Test loss                         Test metric
0         [1.66e-03, 2.07e-03, 9.14e-01]    [1.66e-03, 2.07e-03, 9.14e-01]    []  
1         [3.19e-03, 1.33e-02, 8.00e-01]    [3.19e-03, 1.33e-02, 8.00e-01]    []  

Best model at step 1:
  train loss: 8.16e-01
  test loss: 8.16e-01
  test metric: []

'train' took 75.888453 s

Compiling model...
'compile' took 9.636255 s

Training model...

Step      Train loss                        Test loss                         Test metric
1         [3.19e-03, 1.33e-02, 8.00e-01]    [3.19e-03, 1.33e-02, 8.00e-01]    []  
1000      [1.63e-03, 1.27e-01, 1.21e-02]    [1.63e-03, 1.27e-01, 1.21e-02]        
1359      [7.04e-04, 1.27e-01, 1.16e-02]    [7.04e-04, 1.27e-01, 1.16e-02]    []  

Best model at step 1359:
  train loss: 1.39e-01
  test loss: 1.39e-01
  test metric: []

'train' took 289.741353 s

{'train_geo_start': 0.5066, 'train_geo_end': 0.59, 'train_time_s

In [ ]:
file_name = save_csv(result, overlap)

uploaded results_1715034351_.csv


In [ ]:
file_name

'results_1715034351_.csv'